In [6]:
"""
!python3.7 -m pip install sentencepiece
!python3.7 -m pip install bert-for-tf2
!python3.7 -m pip install tensorflow_hub
"""


    100% |████████████████████████████████| 1.0MB 1.2MB/s eta 0:00:01


In [1]:
import pyspark

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *

import pandas as pd

import re

import random

import socket

import tensorflow as tf
import tensorflow_hub as thub
import bert

import matplotlib.pyplot as plt
%matplotlib inline

/opt/conda/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
bert_layer = thub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_wwm_cased_L-24_H-1024_A-16/1",
                            trainable=True)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
BertTokenizer = bert.bert_tokenization.FullTokenizer
tokenizer = BertTokenizer(vocabulary_file, do_lower_case=False)

In [3]:
config = pyspark.SparkConf().setAll([("spark.dynamicAllocation.enabled","True"),
                                    ("spark.executor.cores","2")])

In [4]:
sc = SparkContext(conf=config)
spark = SparkSession(sc)

In [12]:
%time tsarc = spark.read.csv("gs://sarc-bucket-5/reddit_trunc.csv", inferSchema=True, header=False, sep = ',')

CPU times: user 13.4 ms, sys: 340 µs, total: 13.8 ms
Wall time: 1min 12s


In [13]:
# Rename columns: 

tsarc = tsarc.withColumnRenamed('_c0','label').withColumnRenamed('_c1','subreddit').withColumnRenamed('_c2','context')


In [14]:
tsarc.show(10)

+-----+---------+--------------------+
|label|subreddit|             context|
+-----+---------+--------------------+
|    0| Portland|All these fucking...|
|    0|     milf|Mother of one on ...|
|    0| gonewild|{F}uckable? ;) Th...|
|    0| politics|Took a loan. The ...|
|    0|     pics|I see your kitche...|
|    0| politics|Man, that was pol...|
|    0|  atheism|"Met my first fun...|
|    0|     gifs|Emma Watson danci...|
|    0|    ducks|The Duck is losin...|
|    0|      CFB|I would just like...|
+-----+---------+--------------------+
only showing top 10 rows



In [16]:
strings = "{F}uckable? ;) The dumb peopld yeah 123 !!1"
tokenized = ["[CLS]"] + tokenizer.tokenize(strings) + ["[SEP]"]
ids = tokenizer.convert_tokens_to_ids(tokenized)
ids

[101,
 196,
 143,
 198,
 190,
 2158,
 1895,
 136,
 132,
 114,
 1109,
 14908,
 185,
 8209,
 1643,
 5253,
 8147,
 13414,
 106,
 106,
 122,
 102]

In [21]:
def tokenize_sample(context):
    
    """
    To be applied over dataframe.
    Takes a string and converts it to token IDs via BERT tokenizer,
    adding the necessary beginning and end tokens
    """
    
    tokenized = ["[CLS]"] + tokenizer.tokenize(context) + ["[SEP]"]
    ids = tokenizer.convert_tokens_to_ids(tokenized)
    
    return ids

tokenize_sample_udf = F.udf(tokenize_sample, ArrayType(IntegerType()))

In [24]:
tsarc = tsarc.withColumn("tokens", tokenize_sample_udf(tsarc.context))

In [26]:
tsarc = tsarc.drop('context')
tsarc.show()

+-----+-------------------+--------------------+
|label|          subreddit|              tokens|
+-----+-------------------+--------------------+
|    0|           Portland|[101, 1398, 1292,...|
|    0|               milf|[101, 4872, 1104,...|
|    0|           gonewild|[101, 196, 143, 1...|
|    0|           politics|[101, 6466, 1377,...|
|    0|               pics|[101, 146, 1267, ...|
|    0|           politics|[101, 2268, 117, ...|
|    0|            atheism|[101, 107, 19415,...|
|    0|               gifs|[101, 4913, 7422,...|
|    0|              ducks|[101, 1109, 16627...|
|    0|                CFB|[101, 146, 1156, ...|
|    0|              funny|[101, 1284, 2028,...|
|    0|              funny|[101, 138, 16723,...|
|    0|              funny|[101, 1135, 112, ...|
|    0|          AskReddit|[101, 5749, 1207,...|
|    0|                aww|[101, 107, 107, 1...|
|    0|      todayilearned|[101, 157, 17656,...|
|    0|PoliticalDiscussion|[101, 2082, 7691,...|
|    0|          Ask

In [36]:
tsarc.printSchema()

root
 |-- label: integer (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- tokens: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- tokens2: string (nullable = true)



In [35]:
tsarc = tsarc.withColumn("tokens2",F.col("tokens").cast(StringType()))

In [40]:
tsarc = tsarc.drop('tokens')

In [41]:
tsarc.show(10)

+-----+---------+--------------------+
|label|subreddit|             tokens2|
+-----+---------+--------------------+
|    0| Portland|[101, 1398, 1292,...|
|    0|     milf|[101, 4872, 1104,...|
|    0| gonewild|[101, 196, 143, 1...|
|    0| politics|[101, 6466, 1377,...|
|    0|     pics|[101, 146, 1267, ...|
|    0| politics|[101, 2268, 117, ...|
|    0|  atheism|[101, 107, 19415,...|
|    0|     gifs|[101, 4913, 7422,...|
|    0|    ducks|[101, 1109, 16627...|
|    0|      CFB|[101, 146, 1156, ...|
+-----+---------+--------------------+
only showing top 10 rows



In [43]:
%time tsarc.write.csv('gs://sarc-bucket-5/tokens.csv')
#%time tsarc.write.format("text").option("header", "true").save("tokens.txt")

Py4JJavaError: An error occurred while calling o320.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.sca